In [18]:
#!pip install gTTS

from gtts import gTTS

text_to_say = "Como vai? Fale o comando anotar ou alarme"

language = "pt"

gtts_object = gTTS(text = text_to_say,
                  lang = language,
                  slow = False)

gtts_object.save("/content/gtts.wav")

from IPython.display import Audio

Audio("/content/gtts.wav")

In [36]:
!pip install SpeechRecognition
!pip install playsound
!pip install pyjokes

!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install pyaudio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
libportaudio2 is already the newest version (19.6.0-1.1).
libportaudiocpp0 is already the newest version (19.6.0-1.1).
portaudio19-dev is already the newest version (19.6.0-1.1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=PyAudio-0.2.14-cp310-cp310-linux_x86_64.whl size=63862 sha256=016c840df4b1caba6f7d423a0cd7fe5ea435f60262da0c76b151093dca661dbf
  Stored in directory: /root/.cache/pip/wheels/d6/21/f4/0b51d41ba79e51b16295cbb096ec49f334792814d545b508c5
Successfully built pyaudio


In [47]:
!pip install simpleaudio


In [48]:
import time

In [ ]:
#import section
import speech_recognition as sr
from gtts import gTTS
import os
from datetime import datetime
import playsound
import pyjokes
import pyaudio
from pygame import mixer

import simpleaudio as sa
import numpy as np

def play_alarm():
    frequency = 1000  # Frequência em Hz
    duration = 1000  # Duração em ms
    sample_rate = 44100  # Taxa de amostragem
    t = np.linspace(0, duration / 1000, int(sample_rate * duration / 1000), False)

    # Generate a sine wave
    audio = np.sin(frequency * 2 * np.pi * t)
    # Normalize to 16-bit range
    audio *= 32767 / np.max(np.abs(audio))
    # Convert to 16-bit data
    audio = audio.astype(np.int16)
    # Start playback
    play_obj = sa.play_buffer(audio, 1, 2, sample_rate)
    # Wait for playback to finish before exiting
    play_obj.wait_done()


# Função para gravar conteúdo falado em um arquivo de texto
def save_to_file(text, filename='anotacoes.txt'):
    with open(filename, 'a') as file:
        file.write(text + '\n')
    print(f"Anotação salva em {filename}")

# Função para definir um alarme
def set_alarm(alarm_time):
    current_time = datetime.datetime.now().strftime("%H:%M")
    while current_time != alarm_time:
        current_time = datetime.datetime.now().strftime("%H:%M")
        time.sleep(1)
    print("Alarme ativado!")
    for _ in range(5):
        play_alarm()  # Alarme sonoro

#get mic audio
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        # wait for a second to let the recognizer adjust the
        # energy threshold based on the surrounding noise level
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(said)
        except sr.UnknownValueError:
            speak("Sorry, I did not get that.")
        except sr.RequestError:
            speak("Sorry, the service is not available")
    return said.lower()

#speak converted audio to text
def speak(text):
    tts = gTTS(text=text, lang='en')
    filename = "voice.mp3"
    try:
        os.remove(filename)
    except OSError:
        pass
    tts.save(filename)
    playsound.playsound(filename)


#function to respond to commands
def respond(text):
    if 'exit' in text:
        speak("Até logo")
        exit()
    elif 'anotar' in text:
        speak("O que você gostaria de anotar?")
        note = get_audio()
        save_to_file(note)
    elif 'alarme' in text:
        speak("Para que hora você gostaria de definir o alarme? (Formato HH:MM)")
        alarm_time = get_audio()
        set_alarm(alarm_time)
    else:
        speak("Comando não reconhecido. Tente novamente.")
#play music
def playmusic(song):
    mixer.init()
    mixer.music.load(song)
    mixer.music.play()
#stop music
def stopmusic():
    mixer.music.stop()

#let's try it
#text = get_audio()
#speak(text)
while True:
    print("I am listening...")
    text = get_audio()
    respond(text)